<a href="https://colab.research.google.com/github/410650377/introduction-to-Computers/blob/main/Stock_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from io import StringIO
import requests
import pandas as pd
import pickle

In [ ]:
def create_today_timestamp():
    today = time.strftime("%Y-%m-%d",time.gmtime())
    return int(time.mktime(time.strptime(today, "%Y-%m-%d")))
def create_timestamp_from_today(n):
    today = create_today_timestamp()
    return today + n*24*3600

In [ ]:
def create_tw_stock_info_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df = pd.read_html(res.text)[0]
    df.columns = df.iloc[0]
    df = df.iloc[2:]
    #df = df.reset_index(drop=True)
    new_df = df['有價證券代號及名稱'].str.replace(u'\u3000',' ').str.split(u' ',expand=True)
    new_df = new_df.iloc[0:,0:2]   
    new_df.columns = ["Ticker", "StockName"]
    new_df['Sector'] = df['產業別']
    new_df = new_df.dropna(how='any')
    new_df = new_df.reset_index(drop=True)
    return new_df

In [ ]:
tomorrow_timestamp = create_timestamp_from_today(1)
tomorrow_timestamp

1672099200

In [ ]:
tw_stock_info_df = create_tw_stock_info_list()
tw_stock_info_df

,Ticker,StockName,Sector
0,1101,台泥,水泥工業
1,1102,亞泥,水泥工業
2,1103,嘉泥,水泥工業
3,1104,環泥,水泥工業
4,1108,幸福,水泥工業
...,...,...,...
991,5871A,中租-KY甲特,其他業
992,6592A,和潤企業甲特,其他業
993,8112A,至上甲特,電子通路業
994,9941A,裕融甲特,其他業


In [ ]:
stock_df = pd.DataFrame()
#ticker_list = tw_stock_info_df['Ticker']
ticker_list = tw_stock_info_df['Ticker'].tail(30)

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}

for ticker in ticker_list:
    print('## Info: Download Ticker '+ticker+'!')
    site = "https://query1.finance.yahoo.com/v7/finance/download/"+ticker+".TW?period1=0&period2="+str(tomorrow_timestamp)+"&interval=1d&events=history&crumb=hP2rOschxO0"
    print(site)
    try:
        response = requests.get(site, headers=HEADERS)
        #print(response)
        tmp_df = pd.read_csv(StringIO(response.text))
        tmp_df['Ticker'] = ticker
        stock_df = pd.concat([stock_df,tmp_df],axis=0)
        
    except:
        print('## Warning: Ticker '+ticker+' is failed!')
stock_df = stock_df.reset_index(drop=True)
stock_df = stock_df[['Date','Ticker','Open','High','Low','Close','Adj Close','Volume']]


## Info: Download Ticker 9945!
https://query1.finance.yahoo.com/v7/finance/download/9945.TW?period1=0&period2=1672099200&interval=1d&events=history&crumb=hP2rOschxO0
<Response [200]>
## Info: Download Ticker 9946!
https://query1.finance.yahoo.com/v7/finance/download/9946.TW?period1=0&period2=1672099200&interval=1d&events=history&crumb=hP2rOschxO0
<Response [200]>
## Info: Download Ticker 9955!
https://query1.finance.yahoo.com/v7/finance/download/9955.TW?period1=0&period2=1672099200&interval=1d&events=history&crumb=hP2rOschxO0
<Response [200]>
## Info: Download Ticker 9958!
https://query1.finance.yahoo.com/v7/finance/download/9958.TW?period1=0&period2=1672099200&interval=1d&events=history&crumb=hP2rOschxO0
<Response [200]>
## Info: Download Ticker 1312A!
https://query1.finance.yahoo.com/v7/finance/download/1312A.TW?period1=0&period2=1672099200&interval=1d&events=history&crumb=hP2rOschxO0
<Response [200]>
## Info: Download Ticker 1522A!
https://query1.finance.yahoo.com/v7/finance/downloa

In [ ]:
stock_df

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,2000-01-04,9945,2.082554,2.207508,2.057564,2.207508,1.148496,4751375.0
1,2000-01-05,9945,2.207508,2.353287,2.145031,2.332461,1.213505,7549721.0
2,2000-01-06,9945,2.290810,2.478240,2.290810,2.436589,1.267679,8387512.0
3,2000-01-07,9945,2.436589,2.499065,2.374112,2.436589,1.267679,6220152.0
4,2000-01-10,9945,2.436589,2.457414,2.353287,2.436589,1.267679,2911567.0
...,...,...,...,...,...,...,...,...
42040,2022-12-20,6854,112.500000,112.500000,108.000000,108.000000,108.000000,96033.0
42041,2022-12-21,6854,109.000000,111.000000,108.000000,109.500000,109.500000,51167.0
42042,2022-12-22,6854,109.500000,110.500000,106.500000,106.500000,106.500000,53020.0
42043,2022-12-23,6854,104.000000,105.500000,102.000000,102.000000,102.000000,79000.0
